# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from huggingface_hub import notebook_login
notebook_login()


# for convenience you can create an environment variable containing your hub token as HF_TOKEN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 5.6 MB/s eta 0:00:00a 0:00:01


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [3]:
# Let's test the base model before training
prompt = "What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?"
# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
What is the area of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [4]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
#ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")


ds = load_dataset("GAIR/o1-journey")


README.md:   0%|          | 0.00/746 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/615k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/327 [00:00<?, ? examples/s]

In [5]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

from datasets import load_dataset, DatasetDict
from random import randrange

# Define the prompt template (adapt this to your desired format)
def process_dataset(sample):
    conversations = [
        {"role": "question", "content":sample["question"]},
        {"role": "longCOT", "content":sample["longCOT"]},
        {"role": "answer","content":sample["answer"]}
    ]
    return {"messages":conversations}

# Format each split of the dataset
ds= ds.map(process_dataset)

# Split into train and test
ds = ds['train'].train_test_split(test_size=0.1, seed=42)  # 10% for evaluation

# Print an example to verify
sample = ds['train'][randrange(len(ds["train"]))]
sample



Map:   0%|          | 0/327 [00:00<?, ? examples/s]

{'idx': 660,
 'question': 'Rationalize the denominator of $\\frac{3}{2\\sqrt[3]{5}}$. The answer can be written in the form of $\\frac{A\\sqrt[3]{B}}{C}$, where $A$, $B$, and $C$ are integers, $C$ is positive, and $B$ is not divisible by the cube of any prime. Find $A+B+C$.',
 'answer': ' 38',
 'longCOT': "Let's work through the problem of rationalizing the denominator of \\(\\frac{3}{2\\sqrt[3]{5}}\\) and find the expression in the form \\(\\frac{A\\sqrt[3]{B}}{C}\\), with \\(A\\), \\(B\\), and \\(C\\) as integers, \\(C\\) positive, and \\(B\\) not divisible by the cube of any prime. Our goal is to find \\(A+B+C\\).\n\nInitially, one might think about using the conjugate to rationalize the denominator, a method commonly applied to square roots. However, let's pause and consider this. The denominator here is \\(2\\sqrt[3]{5}\\), involving a cube root, not a square root. So, using a conjugate isn't applicable.\n\nInstead, let's try multiplying the numerator and denominator by \\(\\sqrt[

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [6]:
ds["train"]

Dataset({
    features: ['idx', 'question', 'answer', 'longCOT', 'messages'],
    num_rows: 294
})

In [7]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id="smollm_fintuned",  # Set a unique name for your model
    report_to="none",  # Disable wandb integration

)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/1517198919.py:20: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/294 [00:00<?, ? examples/s]

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [8]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("smolLM_qwq")

Step,Training Loss,Validation Loss
50,1.014700,0.993074
100,0.827500,0.962802
150,0.775500,0.954671
200,0.709400,0.965746
250,0.614200,1.007007
300,0.632600,1.013168
350,0.510500,1.064173
400,0.441300,1.140773
450,0.412000,1.186749
500,0.374100,1.222565


In [10]:
trainer.save_model("smolLM_qwq")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [14]:
# Test the fine-tuned model on the same prompt
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_path = "/kaggle/working/smolLM_qwq"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


# Let's test the base model before training
prompt = "What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=1000)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
What is the area, in square units, of a triangle with vertices at $A(1, 1), B(6, 1), C(3, 7)$?
longCOT
Let’s explore the problem of finding the area, in square units, of a triangle with vertices at \( A(1, 1), B(6, 1), C(3,7) \).

Initially, I considered using the formula for the area of a triangle, which is \( \frac{1}{2} x base \times height \). In this case, \( \frac{1}{2} = 1 x 7 \).

This seemed to make sense at the time, but let’s pause and consider what we know so far. We have the vertices, and our goal is to find the area.

Wait a second, let’s rethink this. The problem specifically asks for the area of a triangle with vertices at \( A(1, 1), B(6, 1), C(3, 7) \). So, the first step is to find the area of \( A(1, 1) \).

To do this, let’s calculate the base. This is \( 1 \). And the height, which is also \( 1 \), is \( 6 \).

Now, the area of a triangle is calculated using the base times the height. Let’s calculate that: \( 1 \ times 6 = 16 \).

So, the area

In [20]:
# Let's test the base model before training
prompt ="Compute $\arcsin 1.$ Express your answer in radians."

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.
outputs = model.generate(**inputs, max_new_tokens=3000)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Compute $rcsin 1.$ Express your answer in radians.
longCOT
Let’s explore the problem of finding the value of \( rcsin 1 \) for which \( r \ is an integer \( > 1 \). We need to find \( r \) such that \( rcsin 1 = 0 \). In other words, we want \( r = 1 \).

Initially, I thought about rewriting \( sin 1 \) in terms of other functions. Let’s see if we can simplify this. 

Initially, \( sin 1 \) is defined as \(\frac{\arcsin 1}{\sqrt{3}} \). So, we can rewrite \( sin 1 \) as \(\frac{\sqrt{2} \arcsin 1}{\sqrt{3}} \). Now, let’s simplify this expression.

\[
\frac{\sqrt{2} \arcsin 1}{\sqrt{3}} = \frac{\sqrt{2} \arcsin 1}{2} = \frac{2\arcsin 1}{1} = 1.
\]

At this point, I realized that I had correctly substituted. Let’s pause and consider what this means. By substituting \(\frac{\sqrt{2} \arcsin 1}{2}\) for \( \sqrt{2} \) in the function \(\arcsin 1\), we substituted back for \( 1 \), leading us to \( 1 = \boxed{\frac{\sqrt{2} \arcsin 1}{2}} \).

This confirms that the v

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.